In [1]:
import re

import gensim
import pandas as pd
import spacy
from gensim import corpora
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words


def remove_short_words(text):
    shortword = re.compile(r"\W*\b\w{1,2}\b")
    return shortword.sub("", text)

In [2]:
from keybert import KeyBERT

en_stop = get_stop_words("en")
en_stop = en_stop.extend(
    [
        "milton",
        "keynes",
        "world",
        "work",
        "uk",
        "support",
        "service",
        "search",
        "contact",
        "shop",
        "usage",
        "business",
        "manage",
        "provide",
        "website",
        "help",
        "product",
        "solution",
        "company",
        "design",
        "get",
        "team",
        "experience",
        "customer",
        "need",
        "client",
        "develop",
        "quick",
        "sale",
        "day",
        "ltd",
        "project",
        "work",
        "office",
        "market",
        "can",
        "will",
        "offer",
        "find",
        "new",
        "year",
        "look",
        "call",
        "see",
        "one",
        "service",
        "news",
        "news",
        "events",
        "press",
        "contact",
        "search",
        "http",
    ]
)

for i in range(10, 30):
    print(i)
    doc = html.loc[html.index[i], "Content"]
    kw_model = KeyBERT()
    print(kw_model.extract_keywords(doc, keyphrase_ngram_range=(1, 1), stop_words=None))

10


NameError: name 'html' is not defined

In [23]:
### Preliminary import and processing of html.csv file
html = pd.read_csv(
    "/Users/cordiez/Documents/CompaniesHouse/OutputFiles/html.csv", index_col=0
)

# Drop irrelevant indexes
idx_to_drop = html[html["Content"].isna() == True].index
html.drop(index=idx_to_drop, inplace=True)
idx_to_drop = html[html["Content"] == "Error"].index
html.drop(index=idx_to_drop, inplace=True)
print(len(html))

### Remove sites shorter than 80 words
# idx_to_drop = html[html['Content'].apply(lambda x:len(x.split()))<80].index
# html.drop(index =idx_to_drop, inplace = True)

print("Number of websites left: ", len(html))

html["Content"] = html["Content"].apply(
    lambda x: re.sub(r"([a-z\.!?])([A-Z])", r"\1 \2", x)
)
html["Content"] = html["Content"].apply(
    lambda x: "".join([i for i in x if not i.isdigit()])
)
html["Content"] = html["Content"].apply(lambda x: remove_short_words(x))
###
tokenizer = RegexpTokenizer(r"\w+")

# create English stop words list
en_stop = get_stop_words("en")
en_stop.extend(
    [
        "milton",
        "keynes",
        "world",
        "work",
        "uk",
        "support",
        "service",
        "search",
        "contact",
        "shop",
        "usage",
        "business",
        "manage",
        "provide",
        "website",
        "help",
        "product",
        "solution",
        "company",
        "design",
        "get",
        "team",
        "experience",
        "customer",
        "need",
        "client",
        "develop",
        "quick",
        "sale",
        "day",
        "ltd",
        "project",
        "work",
        "office",
        "market",
        "can",
        "will",
        "offer",
        "find",
        "new",
        "year",
        "look",
        "call",
        "see",
        "one",
        "service",
        "news",
        "news",
        "events",
        "press",
        "contact",
        "search",
    ]
)

# Create p_stemmer of class PorterStemmer
p_stemmer = PorterStemmer()
tokenizer = RegexpTokenizer(r"\w+")

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


2248
Number of websites left:  2248


In [4]:
# list for tokenized documents in loop
texts = []
doc_set = html["Content"].tolist()
# loop through document list
cpt = 0
for i in doc_set:
    # print(cpt)
    cpt += 1
    # clean and tokenize document string

    ### No capital letters
    raw = i.lower()
    tokens = tokenizer.tokenize(raw)

    # remove stop words from tokens
    stopped_tokens = [i for i in tokens if i not in en_stop]
    stopped_tokens_stemmed = [p_stemmer.stem(i) for i in en_stop]
    # stem tokens
    stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
    stemmed_tokens = [i for i in stemmed_tokens if i not in stopped_tokens_stemmed]
    # add tokens to list
    texts.append(stemmed_tokens)

# turn our tokenized documents into a id <-> term dictionary
dictionary = corpora.Dictionary(texts)

# convert tokenized documents into a document-term matrix
corpus = [dictionary.doc2bow(text) for text in texts]

# generate LDA model
ldamodel = gensim.models.ldamodel.LdaModel(
    corpus, num_topics=36, id2word=dictionary, passes=30
)

In [5]:
ldamodel = gensim.models.ldamodel.LdaModel(
    corpus, num_topics=36, id2word=dictionary, passes=30
)
result = ldamodel.print_topics(num_topics=36, num_words=10)
for elt in result:
    print(elt)
    print("\n")

(0, '0.029*"web" + 0.022*"deliveri" + 0.017*"book" + 0.011*"seo" + 0.009*"email" + 0.009*"airport" + 0.009*"deliv" + 0.008*"arthur" + 0.008*"taxi" + 0.008*"onlin"')


(1, '0.045*"properti" + 0.015*"let" + 0.014*"home" + 0.014*"hous" + 0.010*"mortgag" + 0.009*"profession" + 0.009*"rent" + 0.008*"buckinghamshir" + 0.008*"agent" + 0.008*"bedroom"')


(2, '0.019*"electr" + 0.018*"electrician" + 0.018*"commerci" + 0.016*"legal" + 0.011*"solicitor" + 0.010*"law" + 0.009*"advic" + 0.008*"partner" + 0.008*"employ" + 0.008*"energi"')


(3, '0.034*"children" + 0.031*"tattoo" + 0.017*"foster" + 0.014*"child" + 0.014*"live" + 0.013*"water" + 0.009*"aftercar" + 0.009*"vision" + 0.008*"ink" + 0.008*"famili"')


(4, '0.017*"leo" + 0.017*"nec" + 0.017*"matti" + 0.017*"pulvinar" + 0.016*"luctu" + 0.016*"ullamcorp" + 0.014*"grind" + 0.013*"landscap" + 0.013*"tree" + 0.012*"tellu"')


(5, '0.054*"safeti" + 0.040*"direct" + 0.029*"quot" + 0.025*"messag" + 0.023*"report" + 0.023*"hour" + 0.020*"site" + 0.0

In [292]:
ldamodel = gensim.models.ldamodel.LdaModel(
    corpus, num_topics=30, id2word=dictionary, passes=10
)
result = ldamodel.print_topics(num_topics=30, num_words=10)
for elt in result:
    print(elt)
    print("\n")

(0, '0.014*"home" + 0.010*"recommend" + 0.010*"thank" + 0.008*"time" + 0.008*"hous" + 0.008*"great" + 0.008*"buckinghamshir" + 0.008*"profession" + 0.008*"friendli" + 0.007*"price"')


(1, '0.024*"roof" + 0.018*"skip" + 0.018*"hire" + 0.015*"church" + 0.014*"home" + 0.013*"wast" + 0.011*"centr" + 0.011*"commun" + 0.007*"com" + 0.007*"commerci"')


(2, '0.055*"tune" + 0.029*"fiat" + 0.022*"cabl" + 0.018*"fibr" + 0.013*"print" + 0.012*"sensor" + 0.011*"lead" + 0.010*"punto" + 0.009*"system" + 0.009*"control"')


(3, '0.020*"gift" + 0.016*"art" + 0.013*"wax" + 0.011*"love" + 0.010*"set" + 0.009*"laundri" + 0.009*"soul" + 0.008*"care" + 0.007*"crystal" + 0.007*"bag"')


(4, '0.023*"engin" + 0.020*"door" + 0.019*"recruit" + 0.015*"window" + 0.012*"appli" + 0.012*"now" + 0.011*"industri" + 0.011*"per" + 0.010*"england" + 0.009*"energi"')


(5, '0.078*"clean" + 0.018*"tattoo" + 0.016*"octopu" + 0.012*"floor" + 0.012*"alarm" + 0.012*"cleaner" + 0.011*"use" + 0.011*"fire" + 0.011*"carpet" + 0.0

## Other trial

In [9]:
crunchbase_industries = pd.read_csv(
    "/Users/cordiez/Documents/CompaniesHouse/OutputFiles/crunchbase_industries.csv"
)
html = pd.read_csv(
    "/Users/cordiez/Documents/CompaniesHouse/OutputFiles/html.csv", index_col=0
)
html_cb = pd.read_csv(
    "/Users/cordiez/Documents/CompaniesHouse/OutputFiles/html2.csv", index_col=0
)

idx_to_drop = html_cb[
    (html_cb["Content"].isna() == True) | (html_cb["Content"] == "Error")
].index
html_cb.drop(index=idx_to_drop, inplace=True)

idx_to_drop = html[
    (html["Content"].isna() == True) | (html["Content"] == "Error")
].index
html.drop(index=idx_to_drop, inplace=True)

In [218]:
# Gensim
import gensim
import gensim.corpora as corpora
import numpy as np
import pandas as pd
# spacy for lemmatization
from gensim.models import CoherenceModel
from gensim.utils import simple_preprocess

nlp = spacy.load("en_core_web_sm")

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this

%matplotlib inline

# Enable logging for gensim - optional
import logging

logging.basicConfig(
    format="%(asctime)s : %(levelname)s : %(message)s", level=logging.ERROR
)

import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning)

In [219]:
from nltk.corpus import stopwords

stop_words = stopwords.words("english")
stop_words.extend(
    [
        "milton",
        "keynes",
        "world",
        "work",
        "uk",
        "support",
        "service",
        "search",
        "contact",
        "shop",
        "usage",
        "business",
        "manage",
        "provide",
        "website",
        "help",
        "product",
        "solution",
        "company",
        "design",
        "get",
        "team",
        "experience",
        "customer",
        "need",
        "client",
        "develop",
        "quick",
        "sale",
        "day",
        "ltd",
        "project",
        "work",
        "office",
        "market",
        "can",
        "will",
        "offer",
        "find",
        "new",
        "year",
        "look",
        "call",
        "see",
        "one",
        "service",
        "news",
        "news",
        "events",
        "press",
        "contact",
        "search",
    ]
)

In [236]:
html_cb.drop(index=[311], inplace=True)
data = html_cb.Content.tolist()

data = [re.sub(r"([a-z\.!?])([A-Z])", r"\1 \2", sent) for sent in data]

# Remove Emails
data = [re.sub("\S*@\S*\s?", "", sent) for sent in data]
# Remove new line characters
data = [re.sub("\s+", " ", sent) for sent in data]
# Remove distracting single quotes
data = [re.sub("'", "", sent) for sent in data]
data = [sent.replace("/", " ") for sent in data]

In [237]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield (
            gensim.utils.simple_preprocess(str(sentence), deacc=True)
        )  # deacc=True removes punctuations


data_words = list(sent_to_words(data))

In [238]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(
    data_words, min_count=5, threshold=100
)  # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example

In [239]:
def remove_stopwords(texts):
    return [
        [word for word in simple_preprocess(str(doc)) if word not in stop_words]
        for doc in texts
    ]


def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]


def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]


def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append(
            [token.lemma_ for token in doc if token.pos_ in allowed_postags]
        )
    return texts_out

In [240]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load("en_core_web_sm")

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(
    data_words_bigrams, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]
)

# print(data_lemmatized[:1])

In [241]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in data_lemmatized]

# View
# print(corpus[:1])

In [242]:
lda_model = gensim.models.ldamodel.LdaModel(
    corpus=corpus,
    id2word=id2word,
    num_topics=14,
    passes=20,
    alpha="auto",
    per_word_topics=True,
)

In [243]:
html_cb[html_cb.Content.str.contains("敬请致电")]

,Website,Content


In [244]:
result = ldamodel.print_topics(num_topics=14, num_words=5)
for elt in result:
    print(elt)

(37, '0.018*"media" + 0.016*"digit" + 0.016*"social" + 0.015*"seo" + 0.011*"bed"')
(17, '0.072*"cooki" + 0.028*"use" + 0.019*"necessari" + 0.014*"consent" + 0.014*"function"')
(36, '0.024*"toggl" + 0.017*"clamp" + 0.016*"latch" + 0.014*"engin" + 0.009*"duti"')
(34, '0.018*"read" + 0.016*"technolog" + 0.013*"light" + 0.009*"envis" + 0.009*"grow"')
(2, '0.024*"properti" + 0.017*"pcm" + 0.013*"learn" + 0.011*"let" + 0.009*"add"')
(4, '0.023*"intellig" + 0.019*"leadership" + 0.018*"skill" + 0.015*"qualif" + 0.015*"award"')
(22, '0.034*"safeti" + 0.021*"risk" + 0.020*"fire" + 0.019*"health" + 0.017*"assess"')
(35, '0.041*"energi" + 0.015*"film" + 0.014*"display" + 0.012*"banner" + 0.011*"print"')
(31, '0.047*"bedroom" + 0.023*"apart" + 0.012*"amscan" + 0.009*"knight" + 0.008*"visibl"')
(24, '0.015*"click" + 0.015*"cast" + 0.010*"time" + 0.010*"celestra" + 0.008*"view"')
(6, '0.115*"therapi" + 0.021*"blood" + 0.021*"ozon" + 0.021*"explor" + 0.021*"recommend"')
(7, '0.015*"aculab" + 0.011*"da

In [112]:
px.line(
    x=np.arange(1, 50, 1),
    y=scores,
    labels={"x": "Number of topics", "y": "Topic Coherence Score"},
)

In [302]:
ch = """(0, '0.011*"recruit" + 0.008*"now" + 0.008*"appli" + 0.007*"take" + 0.007*"industri" + 0.007*"retail" + 0.006*"england" + 0.006*"sustain" + 0.006*"per" + 0.006*"back"')


(1, '0.039*"train" + 0.038*"cours" + 0.018*"learn" + 0.009*"student" + 0.008*"level" + 0.007*"workshop" + 0.007*"class" + 0.007*"educ" + 0.007*"skill" + 0.007*"join"')


(2, '0.021*"autom" + 0.017*"notari" + 0.010*"process" + 0.009*"public" + 0.008*"technolog" + 0.008*"document" + 0.007*"industri" + 0.007*"read" + 0.006*"partner" + 0.006*"global"')


(3, '0.022*"deliveri" + 0.018*"boiler" + 0.018*"heat" + 0.015*"ga" + 0.015*"plumb" + 0.014*"courier" + 0.011*"logist" + 0.010*"ship" + 0.009*"quot" + 0.009*"email"')


(4, '0.047*"tune" + 0.038*"abarth" + 0.025*"car" + 0.021*"fiat" + 0.015*"vehicl" + 0.009*"part" + 0.009*"engin" + 0.008*"key" + 0.008*"merced" + 0.008*"audi"')


(5, '0.023*"speaker" + 0.016*"sensor" + 0.014*"power" + 0.014*"research" + 0.013*"cbd" + 0.012*"system" + 0.011*"rate" + 0.009*"smart" + 0.009*"sound" + 0.009*"batteri"')


(6, '0.024*"park" + 0.014*"museum" + 0.011*"safeti" + 0.011*"visit" + 0.011*"comput" + 0.010*"centr" + 0.009*"fit" + 0.009*"tour" + 0.008*"shirt" + 0.008*"yoga"')


(7, '0.045*"properti" + 0.015*"let" + 0.013*"home" + 0.012*"hous" + 0.011*"mortgag" + 0.010*"pcm" + 0.009*"rent" + 0.009*"agent" + 0.008*"bedroom" + 0.008*"buckinghamshir"')


(8, '0.023*"system" + 0.014*"use" + 0.014*"applic" + 0.014*"thermal" + 0.012*"materi" + 0.012*"control" + 0.011*"comput" + 0.011*"univers" + 0.011*"led" + 0.010*"scienc"')


(9, '0.054*"travel" + 0.023*"book" + 0.019*"now" + 0.017*"airlin" + 0.015*"fli" + 0.015*"holiday" + 0.013*"flight" + 0.012*"bum" + 0.011*"pciaw" + 0.011*"air"')


(10, '0.014*"open" + 0.014*"read" + 0.010*"peopl" + 0.008*"health" + 0.008*"covid" + 0.007*"use" + 0.007*"time" + 0.006*"direct" + 0.006*"counsel" + 0.006*"post"')


(11, '0.038*"truck" + 0.029*"forklift" + 0.022*"steel" + 0.017*"conveyor" + 0.013*"stainless" + 0.013*"pallet" + 0.013*"rack" + 0.012*"section" + 0.011*"warehous" + 0.011*"denbigh"')


(12, '0.027*"txm" + 0.023*"frame" + 0.014*"accessori" + 0.014*"decor" + 0.012*"furnitur" + 0.012*"leather" + 0.011*"watch" + 0.011*"top" + 0.010*"tabl" + 0.009*"climb"')


(13, '0.016*"recommend" + 0.014*"thank" + 0.013*"great" + 0.012*"alway" + 0.012*"remov" + 0.011*"friendli" + 0.010*"time" + 0.010*"qualiti" + 0.009*"profession" + 0.009*"home"')


(14, '0.021*"fibr" + 0.019*"tattoo" + 0.014*"treatment" + 0.012*"skin" + 0.012*"patch" + 0.011*"page" + 0.011*"cabl" + 0.011*"facebook" + 0.010*"plu" + 0.010*"voir"')


(15, '0.023*"electr" + 0.015*"electrician" + 0.015*"safeti" + 0.013*"industri" + 0.012*"instal" + 0.010*"mainten" + 0.010*"home" + 0.009*"system" + 0.009*"engin" + 0.009*"commerci"')


(16, '0.074*"view" + 0.035*"slide" + 0.024*"bed" + 0.017*"home" + 0.016*"previou" + 0.016*"next" + 0.015*"bathroom" + 0.013*"diamond" + 0.011*"set" + 0.011*"badminton"')


(17, '0.023*"door" + 0.022*"window" + 0.014*"roof" + 0.012*"price" + 0.011*"glass" + 0.010*"kitchen" + 0.009*"trade" + 0.009*"mug" + 0.008*"doubl" + 0.007*"christma"')


(18, '0.015*"skip" + 0.015*"room" + 0.015*"hotel" + 0.013*"vat" + 0.011*"home" + 0.011*"wast" + 0.010*"space" + 0.009*"present" + 0.008*"hire" + 0.008*"view"')


(19, '0.017*"account" + 0.011*"consult" + 0.010*"tax" + 0.010*"financi" + 0.009*"right" + 0.008*"plan" + 0.008*"make" + 0.008*"home" + 0.008*"email" + 0.007*"use"')


(20, '0.044*"hire" + 0.030*"school" + 0.015*"home" + 0.013*"meet" + 0.012*"wavendon" + 0.011*"plant" + 0.010*"centr" + 0.010*"divis" + 0.010*"facil" + 0.010*"lift"')


(21, '0.044*"care" + 0.017*"home" + 0.016*"treatment" + 0.015*"health" + 0.014*"beauti" + 0.014*"hair" + 0.014*"live" + 0.011*"donat" + 0.010*"visit" + 0.009*"patient"')


(22, '0.030*"church" + 0.028*"job" + 0.026*"commun" + 0.023*"centr" + 0.017*"regist" + 0.016*"saab" + 0.013*"healthcar" + 0.012*"nurs" + 0.010*"doctor" + 0.010*"home"')


(23, '0.014*"music" + 0.013*"elit" + 0.012*"structur" + 0.008*"leo" + 0.008*"nec" + 0.008*"email" + 0.008*"home" + 0.007*"matti" + 0.007*"pulvinar" + 0.007*"use"')


(24, '0.011*"children" + 0.011*"dog" + 0.009*"art" + 0.008*"googl" + 0.007*"ad" + 0.007*"top" + 0.006*"grass" + 0.006*"onlin" + 0.006*"blog" + 0.006*"click"')


(25, '0.021*"legal" + 0.017*"advic" + 0.016*"commerci" + 0.015*"solicitor" + 0.014*"law" + 0.013*"employ" + 0.011*"construct" + 0.010*"partner" + 0.010*"properti" + 0.009*"privat"')


(26, '0.108*"clean" + 0.024*"floor" + 0.022*"carpet" + 0.013*"massag" + 0.013*"cleaner" + 0.013*"blind" + 0.012*"commerci" + 0.010*"belvoir" + 0.009*"treatment" + 0.009*"bodi"')


(27, '0.093*"cooki" + 0.035*"use" + 0.022*"store" + 0.020*"necessari" + 0.020*"function" + 0.016*"set" + 0.014*"consent" + 0.013*"inform" + 0.012*"polici" + 0.011*"privaci"')


(28, '0.074*"add" + 0.044*"cart" + 0.038*"view" + 0.030*"oil" + 0.030*"price" + 0.021*"login" + 0.021*"list" + 0.020*"incens" + 0.017*"pleas" + 0.016*"wish"')


(29, '0.018*"gift" + 0.013*"wed" + 0.012*"home" + 0.011*"love" + 0.010*"set" + 0.010*"wax" + 0.009*"care" + 0.007*"beauti" + 0.006*"walnut" + 0.006*"dress"')


(30, '0.019*"car" + 0.018*"vehicl" + 0.018*"book" + 0.017*"repair" + 0.013*"price" + 0.010*"insur" + 0.008*"make" + 0.008*"mot" + 0.008*"free" + 0.008*"check"')


(31, '0.052*"fit" + 0.022*"bodi" + 0.022*"train" + 0.022*"andi" + 0.019*"class" + 0.018*"total" + 0.017*"sport" + 0.016*"trainer" + 0.014*"membership" + 0.014*"metabol"')


(32, '0.064*"vacuum" + 0.059*"coffe" + 0.039*"pump" + 0.026*"cup" + 0.021*"spare" + 0.021*"machin" + 0.014*"valv" + 0.014*"filter" + 0.009*"special" + 0.009*"print"')


(33, '0.032*"menu" + 0.027*"food" + 0.024*"order" + 0.019*"tea" + 0.017*"book" + 0.015*"restaur" + 0.014*"street" + 0.014*"drink" + 0.013*"park" + 0.010*"pleas"')


(34, '0.016*"system" + 0.015*"engin" + 0.013*"web" + 0.012*"secur" + 0.011*"print" + 0.008*"high" + 0.008*"inspect" + 0.007*"industri" + 0.007*"brand" + 0.007*"control"')


(35, '0.036*"data" + 0.030*"inform" + 0.021*"person" + 0.020*"use" + 0.020*"may" + 0.010*"parti" + 0.009*"privaci" + 0.008*"third" + 0.008*"collect" + 0.008*"process"')"""


liste = re.findall(r'"(.*?)"', ch)
liste
for i in range(36):
    x = liste[i * 10 : i * 10 + 10]
    w = ", ".join(x)
    print(w)

recruit, now, appli, take, industri, retail, england, sustain, per, back
train, cours, learn, student, level, workshop, class, educ, skill, join
autom, notari, process, public, technolog, document, industri, read, partner, global
deliveri, boiler, heat, ga, plumb, courier, logist, ship, quot, email
tune, abarth, car, fiat, vehicl, part, engin, key, merced, audi
speaker, sensor, power, research, cbd, system, rate, smart, sound, batteri
park, museum, safeti, visit, comput, centr, fit, tour, shirt, yoga
properti, let, home, hous, mortgag, pcm, rent, agent, bedroom, buckinghamshir
system, use, applic, thermal, materi, control, comput, univers, led, scienc
travel, book, now, airlin, fli, holiday, flight, bum, pciaw, air
open, read, peopl, health, covid, use, time, direct, counsel, post
truck, forklift, steel, conveyor, stainless, pallet, rack, section, warehous, denbigh
txm, frame, accessori, decor, furnitur, leather, watch, top, tabl, climb
recommend, thank, great, alway, remov, friendli, 

In [303]:
counts = dict()
for i in range(36):
    counts[i] = 0

for elt in corpus:
    tops = ldamodel.get_document_topics(elt)
    topic_max = 0
    maxi = 0
    for t in tops:
        if t[1] > maxi:
            maxi = t[1]
            topic_max = t[0]

    counts[topic_max] += 1

In [306]:
import pandas as pd

pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 500)
pd.set_option("display.width", 1000)

In [329]:
results = pd.DataFrame()
results["Number of documents"] = None
results["Number of documents"] = None

liste = re.findall(r'"(.*?)"', ch)
liste
for i in range(36):
    x = liste[i * 10 : i * 10 + 10]
    w = ", ".join(x)
    results.loc[i, "Terms"] = w
    results.loc[i, "Number of documents"] = counts[i]

results_with_label.style.set_properties(subset=["Terms"], **{"width": "450px"})

,Number of documents,Terms,Supposed activity
0,197,"recruit, now, appli, take, industri, retail, england, sustain, per, back",human resources
1,55,"train, cours, learn, student, level, workshop, class, educ, skill, join",educational institution
2,41,"autom, notari, process, public, technolog, document, industri, read, partner, global",unknown
3,53,"deliveri, boiler, heat, ga, plumb, courier, logist, ship, quot, email",plumbing & heating
4,27,"tune, abarth, car, fiat, vehicl, part, engin, key, merced, audi",vehicule dealer
5,10,"speaker, sensor, power, research, cbd, system, rate, smart, sound, batteri",electronics
6,30,"park, museum, safeti, visit, comput, centr, fit, tour, shirt, yoga",outdoor activities
7,111,"properti, let, home, hous, mortgag, pcm, rent, agent, bedroom, buckinghamshir",real estate & services
8,24,"system, use, applic, thermal, materi, control, comput, univers, led, scienc",engineering
9,15,"travel, book, now, airlin, fli, holiday, flight, bum, pciaw, air",travel agency


In [320]:
# results.to_excel('/Users/cordiez/Documents/CompaniesHouse/OutputFiles/lda_36_topics.xls')

In [328]:
results_with_label = pd.read_excel(
    "/Users/cordiez/Documents/CompaniesHouse/OutputFiles/lda_36_topics.xls", index_col=0
)

## Other attempt

In [251]:
x_ = []
y_ = []

for k in range(1, 50):
    print(k)
    lda_model = gensim.models.ldamodel.LdaModel(
        corpus=corpus,
        id2word=id2word,
        num_topics=k,
        passes=10,
        alpha="auto",
        per_word_topics=True,
    )

    cm = coherence_model_lda = CoherenceModel(
        model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence="c_v"
    )

    coherence = cm.get_coherence()
    print(coherence)
    x_.append(k)
    y_.append(coherence)

1
0.33406803029773907
2
0.29685359854169996
3
0.2880296569101482
4
0.2825036442270326
5
0.3205585129167157
6
0.32235845243656513
7
0.3139779853118898
8
0.3052180979982194
9
0.3198336030525818
10
0.3214069615212848
11
0.36069890879634453
12
0.35000402415824805
13
0.3288600786533149
14
0.33266406492309564
15
0.35589705652128883
16
0.34431568946886426
17
0.3600652505826427
18
0.36502971831756365
19
0.3529122487410385
20
0.36079233422575746
21
0.3101061268498961
22
0.36771830296691893
23
0.352197730384222
24
0.37208447129863
25
0.379840006066778
26
0.3674522551789098
27
0.36367728877964633
28
0.33582380783706406
29
0.3567529013781576
30
0.36203382975601334
31
0.3649204140177195
32
0.3769534392648437
33
0.3694322311284342
34
0.35014821004850494
35
0.3398566845448563
36
0.3784221905672801
37
0.3847589697175918
38
0.3862000285833136
39
0.3942901164092443
40
0.36018830272180863
41
0.3546898345715532
42
0.3661203308455221
43
0.3546078258813018
44
0.3851557195714665
45
0.40027509534332195
46
0.3

In [254]:
px.line(x=x_, y=y_, labels={"x": "Number of topics", "y": "Topic Coherence Score"})

In [9]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary=ldamodel.id2word)
vis

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
16    -0.139956 -0.035811       1        1  8.177601
8     -0.143389 -0.111435       2        1  7.222477
1     -0.157772  0.037209       3        1  6.163925
15    -0.152554 -0.039988       4        1  5.892300
20    -0.116616  0.039142       5        1  5.383632
21    -0.048539  0.155995       6        1  4.947836
31    -0.127326 -0.085136       7        1  4.786961
27    -0.120269  0.044086       8        1  3.900007
12    -0.055005  0.001175       9        1  3.733263
17    -0.083592 -0.127953      10        1  3.706889
13    -0.068033 -0.000267      11        1  3.180731
30    -0.115745  0.000350      12        1  3.001169
29    -0.091625  0.047859      13        1  2.997571
18    -0.068827  0.073188      14        1  2.632632
2     -0.123722 -0.082520      15        1  2.572830
10    -0.027478  0.097056      16        1  2.541192
6      0.152836 -0.098721      17        1  2.312880
26     0.004865  0.136617      18        1  2.282706
32     0.007687 -0.023564      19        1  2.232434
19    -0.067245  0.098598      20        1  2.177838
24    -0.077657 -0.107945      21        1  2.010715
22     0.081283  0.198999      22        1  1.904287
23    -0.036910 -0.008604      23        1  1.891363
34    -0.009090 -0.177502      24        1  1.775398
7      0.023801  0.043816      25        1  1.596778
0     -0.057035  0.032411      26        1  1.542710
33     0.069973 -0.045587      27        1  1.463467
35     0.212877  0.169639      28        1  1.436098
9      0.174951 -0.045328      29        1  1.259869
25     0.159106  0.142098      30        1  1.171369
3      0.034480  0.060527      31        1  0.863043
14     0.196248 -0.050027      32        1  0.828298
5      0.154240 -0.061362      33        1  0.687947
28     0.205242 -0.153606      34        1  0.680682
11     0.228024 -0.091106      35        1  0.647567
4      0.182770 -0.032304      36        1  0.393534, topic_info=          Term         Freq        Total Category  logprob  loglift
389      cooki  4100.000000  4100.000000  Default  30.0000  30.0000
112   properti  2419.000000  2419.000000  Default  29.0000  29.0000
1103     clean  1361.000000  1361.000000  Default  28.0000  28.0000
343     system  2025.000000  2025.000000  Default  27.0000  27.0000
1897      door   951.000000   951.000000  Default  26.0000  26.0000
...        ...          ...          ...      ...      ...      ...
1034     stock    28.638467   510.674328  Topic36  -4.6925   2.6568
136       soon    24.395273   335.386774  Topic36  -4.8528   2.9169
166       well    29.660209  1176.774584  Topic36  -4.6574   1.8570
482       come    21.157722   824.945981  Topic36  -4.9952   1.8744
687       unit    18.636209  1145.877928  Topic36  -5.1221   1.4189

[2467 rows x 6 columns], token_table=       Topic      Freq     Term
term                           
6761       7  0.933384       ab
14692     29  0.998020   abarth
19558      5  0.952681    abort
5634      15  0.988554    abras
6153       1  0.043224   abroad
...      ...       ...      ...
2770      27  0.088361     zero
10364     28  0.982564  zinpifi
12404     31  0.848492    zombi
27139     20  0.978075  zoohaus
14346      3  0.926109   zoopla

[10139 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[17, 9, 2, 16, 21, 22, 32, 28, 13, 18, 14, 31, 30, 19, 3, 11, 7, 27, 33, 20, 25, 23, 24, 35, 8, 1, 34, 36, 10, 26, 4, 15, 6, 29, 12, 5])

In [16]:
bertopics = {
    -1: [
        ("for", 0.008265164321675149),
        ("with", 0.008231263185320623),
        ("are", 0.007993124020813754),
        ("in", 0.00796930927988763),
        ("we", 0.007932255808533718),
        ("your", 0.00777072965758401),
        ("our", 0.007518084423127548),
        ("have", 0.006765938487297426),
        ("be", 0.006494664651061733),
        ("at", 0.006218298483060666),
    ],
    0: [
        ("business", 0.017752723539710462),
        ("services", 0.011923836806484693),
        ("management", 0.011666561020395934),
        ("marketing", 0.011399199130242236),
        ("courses", 0.010949063776995418),
        ("cloud", 0.009989458836740534),
        ("digital", 0.009685804782037009),
        ("development", 0.009443543245108084),
        ("web", 0.007845240623876213),
        ("design", 0.00770037283705406),
    ],
    1: [
        ("car", 0.02905096028581764),
        ("vehicle", 0.022406302177877674),
        ("repairs", 0.015061123886955269),
        ("servicing", 0.01441736565934265),
        ("cars", 0.014370613356053564),
        ("mercedes", 0.01237085132063048),
        ("repair", 0.012011732266191935),
        ("tyres", 0.01090466369098466),
        ("vehicles", 0.010112850739652872),
        ("benz", 0.009147481662828296),
    ],
    2: [
        ("food", 0.021159454524227638),
        ("menu", 0.017730667429885456),
        ("chicken", 0.015151445293628978),
        ("sauce", 0.014388061293090386),
        ("ipsum", 0.011537194143072312),
        ("pizza", 0.010574274268978986),
        ("restaurant", 0.009149350409264682),
        ("amet", 0.00846040160221736),
        ("fried", 0.008450531275623115),
        ("roast", 0.008119305475431096),
    ],
    3: [
        ("business", 0.029545831194513508),
        ("accounting", 0.022971132943806812),
        ("accountants", 0.022436095400736596),
        ("accountancy", 0.019686798155918565),
        ("services", 0.01626939255492918),
        ("accounts", 0.01230561884506264),
        ("businesses", 0.011211016046050626),
        ("bookkeeping", 0.01091936275372247),
        ("clients", 0.010079599336814465),
        ("company", 0.00938323247454696),
    ],
    4: [
        ("recruitment", 0.038575328278427974),
        ("candidates", 0.018784396671520678),
        ("jobs", 0.018073671172207276),
        ("job", 0.016779486566090192),
        ("hr", 0.012972638654629927),
        ("candidate", 0.012563007395200302),
        ("manager", 0.010303239615886547),
        ("recruit", 0.009511164081352153),
        ("employee", 0.00891903771115337),
        ("vacancies", 0.008060095378105587),
    ],
    5: [
        ("club", 0.014904478089935816),
        ("fitness", 0.01304187637351485),
        ("play", 0.012367306256806853),
        ("classes", 0.011364447408064334),
        ("training", 0.009990725544477917),
        ("open", 0.00980741824918325),
        ("mk", 0.00905021320716155),
        ("membership", 0.008761447248033797),
        ("class", 0.008501709580121735),
        ("sessions", 0.008187719796380801),
    ],
    6: [
        ("property", 0.042128780369251086),
        ("rent", 0.018119992997166507),
        ("keynes", 0.016859209547142255),
        ("valuation", 0.014798406453420728),
        ("estate", 0.013468727575525178),
        ("landlords", 0.009460343349867796),
        ("agent", 0.008954650781515902),
        ("tenants", 0.008866757020705347),
        ("buckinghamshire", 0.008851477877349571),
        ("landlord", 0.00821480243586678),
    ],
    7: [
        ("engineering", 0.01642087615597742),
        ("machining", 0.01233577349410127),
        ("inspection", 0.012182663359267454),
        ("sensors", 0.010624139783438605),
        ("machines", 0.010250876465092673),
        ("aluminium", 0.009800905982534032),
        ("manufacturing", 0.009670107401294696),
        ("automation", 0.009111772072031732),
        ("forklift", 0.008724315994314153),
        ("manufacture", 0.008409064379442839),
    ],
    8: [
        ("shop", 0.024303020273587737),
        ("casino", 0.018499870156486315),
        ("lehenga", 0.015892563663842695),
        ("wedding", 0.013269020176675526),
        ("options", 0.012620027758860434),
        ("select", 0.012569769384774625),
        ("accessories", 0.012253039568975447),
        ("clothing", 0.010604783032803475),
        ("brands", 0.009173172338994715),
        ("mens", 0.00881634437145051),
    ],
    9: [
        ("paving", 0.0350352189636455),
        ("homes", 0.01315061928699601),
        ("slabs", 0.013090586024468045),
        ("sandstone", 0.012411280929075171),
        ("plastering", 0.01202158378541018),
        ("building", 0.01198181309602571),
        ("limestone", 0.01131616790592148),
        ("granite", 0.01073222720300931),
        ("construction", 0.010721306626089798),
        ("decorating", 0.009285793003481398),
    ],
    10: [
        ("hours", 0.019567051121228352),
        ("abortion", 0.0189516896498),
        ("contact", 0.016529886751308218),
        ("time", 0.0158983179887058),
        ("call", 0.01563622084478462),
        ("agoreport", 0.01471995149212819),
        ("nowget", 0.012053531317850658),
        ("bpas", 0.010891994314565393),
        ("appointment", 0.010530474138094211),
        ("01908", 0.010441068621411967),
    ],
    11: [
        ("care", 0.0848218266221271),
        ("home", 0.022201568806337512),
        ("support", 0.019543021822696925),
        ("living", 0.018795199556597923),
        ("services", 0.015031187563243112),
        ("service", 0.011069082474480503),
        ("provide", 0.009503931161254364),
        ("respite", 0.009093570728189941),
        ("live", 0.00848089277474008),
        ("domiciliary", 0.008345682435490456),
    ],
    12: [
        ("heating", 0.06923329718663297),
        ("plumbing", 0.0621638831060798),
        ("gas", 0.05331877698692185),
        ("boiler", 0.05206089324597366),
        ("boilers", 0.01568735074332285),
        ("repairs", 0.013024180059626087),
        ("servicing", 0.011893669610922149),
        ("emergency", 0.010622562352328377),
        ("plumber", 0.010506308823134074),
        ("ventilation", 0.010279607904494031),
    ],
    13: [
        ("cleaning", 0.10323987684394824),
        ("clean", 0.029055732263025),
        ("window", 0.014805031523450892),
        ("carpet", 0.014476212330094084),
        ("services", 0.013122097812024556),
        ("office", 0.011263305418390122),
        ("cleaners", 0.011223390654234899),
        ("service", 0.010156827753375276),
        ("cleaner", 0.009962747312885593),
        ("home", 0.009335365807245356),
    ],
    14: [
        ("personal", 0.02307317155187561),
        ("vacuum", 0.019688659358249695),
        ("hotels", 0.01834992213500955),
        ("cookies", 0.012501156814693623),
        ("home", 0.012067027882640545),
        ("privacy", 0.010422868044391314),
        ("information", 0.010392010176633099),
        ("web", 0.008911738420827665),
        ("kitchen", 0.008795687333939532),
        ("furniture", 0.008644796187355181),
    ],
    15: [
        ("courses", 0.03431287727500996),
        ("course", 0.024963719030388944),
        ("training", 0.02170795078283909),
        ("cpfi", 0.01591245861600646),
        ("presenter", 0.01507890667409569),
        ("counselling", 0.01330085514012069),
        ("skills", 0.012669965783777106),
        ("psychotherapy", 0.011454837208970132),
        ("therapy", 0.009546062063878275),
        ("ctec", 0.009197859527832732),
    ],
    16: [
        ("de", 0.1858508398007789),
        ("et", 0.0496594273864133),
        ("la", 0.03314338163216213),
        ("des", 0.032617908981687496),
        ("und", 0.03025448338354207),
        ("facebook", 0.02962684953600769),
        ("en", 0.024959575257838314),
        ("les", 0.024909343225369456),
        ("publications", 0.02102705577644125),
        ("oubliees", 0.019486415773998036),
    ],
    17: [
        ("doors", 0.06310186170616343),
        ("windows", 0.04638388640038314),
        ("door", 0.032496598837384),
        ("pvc", 0.01815484722803936),
        ("window", 0.016700040550595523),
        ("aluminium", 0.012497582109835348),
        ("showroom", 0.011857431041898955),
        ("blinds", 0.010482228864044553),
        ("riser", 0.010304756012306291),
        ("design", 0.009103437714650274),
    ],
    18: [
        ("law", 0.04807164694651549),
        ("solicitors", 0.039647515822436676),
        ("legal", 0.030383817073326592),
        ("mediation", 0.01750902097722904),
        ("clients", 0.014715724916190597),
        ("business", 0.013134267891792414),
        ("disputes", 0.011707735825864672),
        ("litigation", 0.010961919803938031),
        ("services", 0.010195506155546166),
        ("dispute", 0.009452627217145706),
    ],
    19: [
        ("church", 0.042879456404928615),
        ("prayer", 0.02417694194659067),
        ("community", 0.015137899073614802),
        ("christian", 0.013619153263258677),
        ("nursery", 0.013501922735046804),
        ("worship", 0.013027473872075273),
        ("ministry", 0.010926268408837326),
        ("children", 0.010871186659817693),
        ("preschool", 0.010686586531907643),
        ("fellowship", 0.009972585415811617),
    ],
    20: [
        ("fire", 0.0783082554173926),
        ("security", 0.05683168047881238),
        ("safety", 0.0324194923964538),
        ("alarms", 0.020615702037470873),
        ("alarm", 0.019259512428561183),
        ("guards", 0.01479879652739567),
        ("emergency", 0.013680826373789007),
        ("extinguishers", 0.012132540684980271),
        ("lighting", 0.00975683438841728),
        ("protection", 0.008797997590947643),
    ],
    21: [
        ("website", 0.055916306507423),
        ("hosting", 0.04479414470902204),
        ("web", 0.03756869219493389),
        ("chess", 0.034893610607495046),
        ("virtualization", 0.03266635234000461),
        ("site", 0.030229963831876646),
        ("cloud", 0.02728018078003669),
        ("domain", 0.027113729419752153),
        ("dns", 0.018603036270556247),
        ("servers", 0.01853058476567347),
    ],
    22: [
        ("electrical", 0.12507307866398512),
        ("electrician", 0.053752617219357354),
        ("electricians", 0.036962835165367934),
        ("industrial", 0.017973581579547898),
        ("lighting", 0.014873250278524848),
        ("contractors", 0.014848071130619443),
        ("electrics", 0.014280725991016688),
        ("installations", 0.014052740938613247),
        ("maintenance", 0.01327853765016459),
        ("alarms", 0.012043921197448114),
    ],
    23: [
        ("massage", 0.035592363789995335),
        ("pain", 0.02681866793096341),
        ("therapy", 0.02047112182177472),
        ("treatments", 0.017581395006588948),
        ("clinic", 0.017355324303655717),
        ("treatment", 0.017293351309193947),
        ("physiotherapy", 0.0160767019807487),
        ("care", 0.014094143956428936),
        ("pilates", 0.012991249016707505),
        ("acupuncture", 0.011263624906891052),
    ],
    24: [
        ("logistics", 0.05738994161024745),
        ("freight", 0.03332898403084088),
        ("transport", 0.029029301626167285),
        ("courier", 0.027324965440176086),
        ("delivery", 0.01834057881248778),
        ("seko", 0.01617423815868375),
        ("parcel", 0.01468563555789507),
        ("shipping", 0.01413277671969003),
        ("express", 0.012328052670289526),
        ("cargo", 0.009534392371986089),
    ],
    25: [
        ("incense", 0.039893249097380026),
        ("event", 0.033489950443724374),
        ("wish", 0.023344713025939833),
        ("art", 0.02167802194127967),
        ("169", 0.01795150684311819),
        ("events", 0.017626263337709367),
        ("memorabilia", 0.015923629203005614),
        ("shop", 0.013530419922445828),
        ("price", 0.013166100017208067),
        ("select", 0.011473091431259078),
    ],
    26: [
        ("skin", 0.0736038093226364),
        ("treatments", 0.048209794098915904),
        ("treatment", 0.03257508160984871),
        ("colour", 0.020993210190352676),
        ("facial", 0.01874806916123882),
        ("brow", 0.015194604206453261),
        ("facials", 0.014864074279808407),
        ("acne", 0.012558646629704469),
        ("peel", 0.011536828498944732),
        ("surgical", 0.011087688314429614),
    ],
    27: [
        ("energy", 0.06375969701368057),
        ("etap", 0.027885036562317705),
        ("power", 0.01855000034547572),
        ("management", 0.016872418213434836),
        ("carbon", 0.016753162402472436),
        ("charging", 0.016147475679542198),
        ("adr", 0.014535172615277795),
        ("renewable", 0.009141606526213764),
        ("complaint", 0.008944257010554737),
        ("evbox", 0.008336976664636822),
    ],
    28: [
        ("mortgage", 0.08787108901104308),
        ("mortgages", 0.06274118858380094),
        ("financial", 0.03960546304486719),
        ("insurance", 0.015444400011322804),
        ("regulated", 0.014506226870615464),
        ("debt", 0.013974436368877513),
        ("remortgage", 0.013234899512175343),
        ("finance", 0.012632920744413223),
        ("loans", 0.012024180234854137),
        ("authorised", 0.012002432329248209),
    ],
    29: [
        ("hair", 0.06987176003039307),
        ("salon", 0.04396445811369011),
        ("headonist", 0.022778864596169457),
        ("treatments", 0.020258382945802622),
        ("tanning", 0.01951673914945296),
        ("hairdressing", 0.016592718455009193),
        ("waxing", 0.01628743379705289),
        ("hairdressers", 0.013866298686250878),
        ("appointment", 0.011462697889687163),
        ("nail", 0.010739346399312),
    ],
    30: [
        ("data", 0.054033431102629446),
        ("printing", 0.048056462525805234),
        ("print", 0.03714257922366373),
        ("controller", 0.024082800741447596),
        ("processing", 0.019836421250739752),
        ("printers", 0.013686301474422836),
        ("banners", 0.011603661784831788),
        ("website", 0.011089466618348967),
        ("information", 0.010665682509773816),
        ("pinterest", 0.008836509173253437),
    ],
    31: [
        ("gifts", 0.04716345939543966),
        ("storage", 0.024381976173723254),
        ("delivery", 0.018430752939662995),
        ("christmas", 0.01729669150865513),
        ("boxes", 0.016229685091846916),
        ("mail", 0.014624224798140344),
        ("parcel", 0.011872531387459827),
        ("marquees", 0.011755334029239261),
        ("mailbox", 0.011159631543029604),
        ("birthdays", 0.010318795936620802),
    ],
    32: [
        ("room", 0.024844176681444664),
        ("stables", 0.02030732380612028),
        ("rooms", 0.01770388115767906),
        ("manor", 0.01636688517447601),
        ("centre", 0.015853493295633847),
        ("guests", 0.01488296616208044),
        ("hotel", 0.014841671979474143),
        ("facilities", 0.01388982283582044),
        ("farmhouse", 0.011628370725077662),
        ("accommodation", 0.010017886627997592),
    ],
    33: [
        ("taxi", 0.059974848219175904),
        ("airport", 0.05404070353371633),
        ("taxis", 0.03615297913173001),
        ("transfers", 0.026989542384159237),
        ("van", 0.025287537012744096),
        ("vehicles", 0.015899652403411603),
        ("cab", 0.013444679254340395),
        ("minibus", 0.012908387774403654),
        ("travel", 0.012899633838085777),
        ("cars", 0.01287810420412975),
    ],
    34: [
        ("construction", 0.04278189380693605),
        ("asphalt", 0.026221677441832434),
        ("geotechnical", 0.0227690057135629),
        ("engineering", 0.018736682967454385),
        ("concrete", 0.013799549303720264),
        ("maintenance", 0.012196322485679242),
        ("geo", 0.011855739199607656),
        ("rhinophalt", 0.011630500737660828),
        ("contracts", 0.010898910139141826),
        ("groundworks", 0.010689516637359784),
    ],
    35: [
        ("school", 0.07463273031560747),
        ("malvern", 0.03901216587136035),
        ("academy", 0.02483273207288295),
        ("children", 0.016298101152754107),
        ("learn", 0.014920593934286236),
        ("brighton", 0.014016465747062737),
        ("admissions", 0.01361776392296791),
        ("education", 0.01345325829128608),
        ("hazard", 0.013059339578908124),
        ("london", 0.012350908556836793),
    ],
    36: [
        ("support", 0.037951935011750086),
        ("children", 0.02818513805546341),
        ("charity", 0.024527921325733826),
        ("donate", 0.0185598139115708),
        ("ymca", 0.018136922645200046),
        ("adoption", 0.01545730232599146),
        ("families", 0.012775056456987706),
        ("child", 0.012172643222529098),
        ("stories", 0.011178174399864243),
        ("shops", 0.01062969394840288),
    ],
    37: [
        ("airlines", 0.06614193032955898),
        ("airways", 0.03172482737553346),
        ("travel", 0.021298455214843005),
        ("cart", 0.0201217926138773),
        ("umrah", 0.018892233361417602),
        ("holidays", 0.01707181842364157),
        ("snoring", 0.01662825406056967),
        ("departure", 0.013030861980288681),
        ("flights", 0.012325622188856427),
        ("seller", 0.012020126116666283),
    ],
    38: [
        ("dog", 0.10652763408009985),
        ("dogs", 0.0736710223064156),
        ("breeding", 0.02954480979113045),
        ("pet", 0.022478206187882026),
        ("groomers", 0.02000273766234916),
        ("grooming", 0.01682046598681605),
        ("puppy", 0.014119579526364112),
        ("odour", 0.012366015421314125),
        ("pets", 0.012268892772571592),
        ("insemination", 0.011652410563937432),
    ],
    39: [
        ("beer", 0.06756895450433012),
        ("brewery", 0.030542918453201208),
        ("ale", 0.029740285989473685),
        ("beers", 0.017579750821537753),
        ("brewer", 0.01710477590973195),
        ("keg", 0.015851957280198014),
        ("ales", 0.01385162948966265),
        ("brewing", 0.013425396568522676),
        ("ipa", 0.011666179333682034),
        ("brewed", 0.011666179333682034),
    ],
    40: [
        ("roof", 0.09881228814521657),
        ("roofing", 0.09742028373093221),
        ("lanterns", 0.043029365542108865),
        ("lantern", 0.03605226245921931),
        ("roofs", 0.03096947801642926),
        ("rooflights", 0.021028036374296767),
        ("chimney", 0.018872259514844173),
        ("skylights", 0.015123100857233437),
        ("light", 0.012639095143277095),
        ("rooflight", 0.011562744173063345),
    ],
    41: [
        ("script", 0.2784624369978815),
        ("java", 0.27715516998405),
        ("enable", 0.21374940315836263),
        ("browser", 0.16770516612693664),
        ("frames", 0.0757900287241054),
        ("loading", 0.05551741271918407),
        ("grainger", 0.05093217605916233),
        ("binzy", 0.05093217605916233),
        ("magento", 0.04413661546543737),
        ("alfa", 0.03799562052183263),
    ],
}


def return_topics(bert_output):
    result = []
    for k in bert_output.keys():
        ch = ""
        for elt in bert_output[k]:
            ch = ch + elt[0] + ", "
        ch = ch[:-2]
        result.append(ch)
    return result


return_topics(bertopics)

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['for, with, are, in, we, your, our, have, be, at',
 'business, services, management, marketing, courses, cloud, digital, development, web, design',
 'car, vehicle, repairs, servicing, cars, mercedes, repair, tyres, vehicles, benz',
 'food, menu, chicken, sauce, ipsum, pizza, restaurant, amet, fried, roast',
 'business, accounting, accountants, accountancy, services, accounts, businesses, bookkeeping, clients, company',
 'recruitment, candidates, jobs, job, hr, candidate, manager, recruit, employee, vacancies',
 'club, fitness, play, classes, training, open, mk, membership, class, sessions',
 'property, rent, keynes, valuation, estate, landlords, agent, tenants, buckinghamshire, landlord',
 'engineering, machining, inspection, sensors, machines, aluminium, manufacturing, automation, forklift, manufacture',
 'shop, casino, lehenga, wedding, options, select, accessories, clothing, brands, mens',
 'paving, homes, slabs, sandstone, plastering, building, limestone, granite, construction, de

In [13]:
"script, java, enable, browser, frames, loading, grainger, binzy, magento, alfa, "

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


'script, java, enable, browser, frames, loading, grainger, binzy, magento, alfa'

In [22]:
for i in html.index:
    if " la " in html.loc[i, "Content"]:
        print(i)

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [24]:
html.Content

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0       Milton Keynes Taxi Service | Cab Hire | Airpor...
1        Support, Services Consultancy Milton Keynes S...
2       Milton Keynes Chartered Accountants | Jenner C...
3        Call Roofing Milton Keynes - EXPERTS ROOFING ...
4        Call Surveys Call Surveys Email: info@stcalls...
                              ...                        
2716    Zen Shots Studio Photography | Milton Keynes &...
2717    Zeroshift – Advanced Transmission Technologies...
2718    Zexfret. Congo Shipping Service Air and Sea. C...
2719    Takeaway Menu for Takeaways and Restaurants - ...
2720    Welcome Zoohause Java Script seems disabled yo...
Name: Content, Length: 2248, dtype: object